In [1]:
import sys
sys.path.append('/Users/zhengj10/Python/Modules')
import h5py
import os
import numpy as np
from skimage.io import imsave
import matplotlib.pyplot as plt
import yaml
import pandas as pd
import PatcherBot as pb
from glob import glob
import seaborn as sns
import tifffile as tff


In [2]:
columns = ['Date','Plate','Well','Cell','Mutation','Transfected_date','Leak','Ra','dFF_p','dFF_s','F0',
           'amp_on_single',
           'amp_off_single','tau_on_single','tau_off_single','bleach_x','bleach','amp_on_f_double',
           'amp_on_s_double','amp_off_f_double','amp_off_s_double','tau_on_f_double','tau_off_f_double',
           'tau_on_s_double','tau_off_s_double','time_on_10-90','time_off_10-90','Leak_min','Ra_max','Objective']
df = pd.DataFrame(columns = columns)
df


,Date,Plate,Well,Cell,Mutation,Transfected_date,Leak,Ra,dFF_p,dFF_s,...,amp_off_s_double,tau_on_f_double,tau_off_f_double,tau_on_s_double,tau_off_s_double,time_on_10-90,time_off_10-90,Leak_min,Ra_max,Objective


### Load params from .mat file

In [ ]:
# sumPath = '/Volumes/genie/Jihong_GENIE_stuff/PatcherBot/summary'
# imgPath = '/Volumes/genie/BenArthur/patch-screen/bot'
# rawPath = '/Volumes/genie/Jihong_GENIE_stuff/PatcherBot/raw'
sumPath = '/Users/zhengj10/Desktop/Voltron2/iclamp'
imgPath = '/Volumes/Backup Plus/patch-screen/bot'
rawPath = '/Volumes/Backup Plus/PatcherBot/raw'
os.path.exists(imgPath)


In [ ]:
# datePath = pb.getPathInfo(imgPath)
datePath = ['2020.07.29_000','2020.07.30_000',
            '2020.08.11_000','2020.08.12_000','2020.08.13_000']
# datePath = datePath[9:]
datePath


In [ ]:
# df = pd.read_pickle(sumPath + '/summary_sel.pkl')
# # df.index = np.arange(1,len(df)+1)
# print(len(df))
# # df = df.drop(df.index[69:161])
# # print(len(df))
# df

In [ ]:
# idx_del = df[(df['Mutation'] == '421.1')].index
# idx_del
# df.drop(idx_del[:6], inplace=True)
# df.replace('421.1','421.1_steady',inplace=True)
# df[df.Mutation == '421.1_steady']

In [ ]:
df_sel = [df['Mutation'],df['dFF']]
df_sel = pd.DataFrame(df_sel)
df_sel = df_sel.transpose()
df_sel
# col = ['Mutation','dFF']


In [ ]:
df_sel = df

In [ ]:
data_all={}
for param in(df_sel.columns):
    data_all[param] = []
for date in datePath:
    platePath = pb.getPathInfo(os.path.join(imgPath,date))
    for plate in platePath:
        cellPath = pb.getPathInfo(os.path.join(imgPath, date, plate))
        for cell in(cellPath):
            print(cell)
            matFiles = glob(os.path.join(imgPath, date, plate, cell) + '/*.mat')
            if len(matFiles) < 9:
                print('bad trial: ' + '/' + date + '/' + plate + '/' + cell)
                continue
            dFF_p = []
            for trial in range(9):
                matFile = os.path.join(imgPath,date,plate,cell) + '/' + str(trial).zfill(3) + '.mat'
                f = h5py.File(matFile,'r')
                d = f['params']['fit_pix_alpha_single']
                st = d[0]
                obj = f[st]
                dff_p = obj['bkg_norm_ave_dFoF0_transient'][()][0]
                dFF_p.append(dff_p)
            data_all['Mutation'].append('421.1_peak')
            data_all['dFF'].append(dFF_p)

In [ ]:
df_new = pd.DataFrame(data_all, columns = list(df_sel))
df_new

In [ ]:
df_sel = df_sel.append(df_new, ignore_index = True)
df_sel.index = np.arange(1, len(df_sel)+1)
df_sel

In [ ]:
import unicodedata
import seaborn as sns
sns.set()
sns.set_style(style='whitegrid')

fig = plt.figure(figsize=[6,5])
params = ['dFF']
x_label = 'mV'
y_label = [unicodedata.lookup("GREEK CAPITAL LETTER delta") + 'F/F']



labels = {
            '421.1_peak':'Voltron_peak',
            '421.1_steady':'Voltron_steady',
            '421.2096':'Voltron A122D',
            '421.4909':'Voltron N69E R78H A122D V196P',
            '421.4919':'Voltron R78H A122D V196P',
            '421.4876':'Voltron N69E A122D',
            '421.4892':'Voltron N69E V196P'
    
         }



for i in range(len(params)):

    ax = plt.subplot(1,1,i+1)

    for sensor in ['421.1_peak','421.1_steady','421.2096','421.4909','421.4919','421.4876','421.4892']: 

        df_sensor = df_sel[df_sel.Mutation == sensor]
        param_mean = np.nanmean(df_sensor[params[i]].tolist(), axis=0)
        param_sem = np.nanstd(df_sensor[params[i]].tolist(), axis=0)/(len(df_sensor)-1)
        x = np.linspace(-100,60,9)
        ax.errorbar(x, param_mean, yerr = param_sem, fmt = 'o-',
                        label = labels[sensor]+' (%d)' %len(df_sensor))
    ax.legend()
    ax.set_xlabel(x_label)
#     ax.set_ylim([-0.3,0.3])
    ax.set_ylabel(y_label[i])
#     ax.set_title(params[i])


handles, labels = ax.get_legend_handles_labels() # Following 3 lines remove the errorbars from legend
handles = [h[0] for h in handles]
ax.legend(handles, labels, loc='lower left', bbox_to_anchor=(0.5, 0.6))

plt.setp(ax, xticks=x)
plt.tight_layout()
# plt.legend()
# fig.savefig('/Users/zhengj10/Desktop/Voltron_variants_dFF.png',dpi=1200)
plt.show()


In [ ]:
# ## Update well info
# for i,r in df.iterrows():
#     if os.path.exists(os.path.join(rawPath,r['Date'],r['Plate'],r['Cell'],'camera_clamp1_000')):
#         df.loc[i,'Well'] = 'well_1'
#     elif os.path.exists(os.path.join(rawPath,r['Date'],r['Plate'],r['Cell'],'camera_clamp2_000')):
#         df.loc[i,'Well'] = 'well_2'
#     elif os.path.exists(os.path.join(rawPath,r['Date'],r['Plate'],r['Cell'],'camera_clamp3_000')):
#         df.loc[i,'Well'] = 'well_3'
#     elif os.path.exists(os.path.join(rawPath,r['Date'],r['Plate'],r['Cell'],'camera_clamp4_000')):
#         df.loc[i,'Well'] = 'well_4'


In [ ]:
# data_all={}
# for param in(df.columns):
#     data_all[param] = []
# for date in datePath:
#     platePath = pb.getPathInfo(os.path.join(rawPath, date))
#     for plate in platePath:
#         idxFile = os.path.join(rawPath, date, plate, '.index')
#         meta = yaml.load(open(idxFile), Loader=yaml.FullLoader)
        
#         cellPath = pb.getPathInfo(os.path.join(imgPath, date, plate))
#         for cell in(cellPath):
#             matFiles = glob(os.path.join(imgPath, date, plate, cell) + '/*.mat')
#             if len(matFiles) < 9:
#                 print('bad trial: ' + '/' + date + '/' + plate + '/' + cell)
#                 continue
#             if os.path.exists(os.path.join(rawPath,date,plate,cell,'camera_clamp1_000')):
#                 dataPath = os.path.join(rawPath,date,plate,cell,'camera_clamp1_000')
#                 data_all['Well'].append('well_1')
#                 mutation = meta['.']['mutation_1']
#                 note = meta['.']['notes'][0:6]
#             elif os.path.exists(os.path.join(rawPath,date,plate,cell,'camera_clamp2_000')):
#                 dataPath = os.path.join(rawPath,date,plate,cell,'camera_clamp2_000')
#                 data_all['Well'].append('well_2')
#                 mutation = meta['.']['mutation_2']
#                 note = meta['.']['notes'][0:6]
#             elif os.path.exists(os.path.join(rawPath,date,plate,cell,'camera_clamp3_000')):
#                 dataPath = os.path.join(rawPath,date,plate,cell,'camera_clamp3_000/')
#                 data_all['Well'].append('well_3')
#                 mutation = meta['.']['mutation_3']
#                 note = meta['.']['notes'][0:6]
#             elif os.path.exists(os.path.join(rawPath,date,plate,cell,'camera_clamp4_000/')):
#                 dataPath = os.path.join(rawPath,date,plate,cell,'camera_clamp4_000')
#                 data_all['Well'].append('well_4')
#                 mutation = meta['.']['mutation_4']
#                 note = meta['.']['notes'][0:6]
            
#             data_all['Date'].append(date)
#             data_all['Plate'].append(plate)
#             data_all['Cell'].append(cell)
#             data_all['Mutation'].append(mutation)
#             data_all['Transfected_date'].append(note[:6])
#             leaks = []
#             Ra_s = [] # List of Ra
#             F0 = []

#             dff=[]
#             amp_on = []
#             amp_off = []
#             tau_on = []
#             tau_off = []

#             amp_on_fast = []
#             amp_on_slow = []
#             amp_off_fast = []
#             amp_off_slow = []
#             tau_on_fast = []
#             tau_on_slow = []
#             tau_off_fast = []
#             tau_off_slow = []
#             time_on = []
#             time_off = []
#             bleach_x = []
#             bleach = []
            
#             for trial in range(9):
#                 ivFile = os.path.join(dataPath,str(trial).zfill(3),'Camera','frames.ma')
#                 leak = pb.get_leak(ivFile)
#                 Ra = pb.get_Ra(ivFile,trial)
#                 matFile = os.path.join(imgPath,date,plate,cell) + '/' + str(trial).zfill(3) + '.mat'
#                 data = pb.get_data(matFile)
#                 f0_single = data['f0']
                                
#                 leaks.append(leak)
#                 Ra_s.append(Ra)
#                 F0.append(f0_single)

#                 trace_single = data['dff_corrected_single']               # for single fit
#                 dff_single = pb.get_dff(trace_single)
#                 amp_on_single = data['amp_on'][0]
#                 amp_off_single = data['amp_off'][0]
#                 tau_on_single = data['tau_on'][0]
#                 tau_off_single = data['tau_off'][0]
#                 bleach_x1 = data['bleach_x'][0]
#                 bleach_x2 = data['bleach_x'][1]
#                 bleach_single_1 = data['bleach'][0]
#                 bleach_single_2 = data['bleach'][1]

#                 dff.append(dff_single)
#                 amp_on.append(amp_on_single)
#                 amp_off.append(amp_off_single)
#                 tau_on.append(tau_on_single)
#                 tau_off.append(tau_off_single)
#                 bleach_x.append(bleach_x1)
#                 bleach_x.append(bleach_x2)
#                 bleach.append(bleach_single_1)
#                 bleach.append(bleach_single_2)

#                 trace_double = data['dff_corrected_double']                # for double fit
#                 amp_on_fast_single = data['amp_on_fast'][0]                
#                 amp_on_slow_single = data['amp_on_slow'][0]
#                 amp_off_fast_single = data['amp_off_fast'][0]
#                 amp_off_slow_single = data['amp_off_slow'][0]
#                 tau_on_fast_single = data['tau_on_fast'][0]
#                 tau_on_slow_single = data['tau_on_slow'][0]
#                 tau_off_fast_single = data['tau_off_fast'][0]
#                 tau_off_slow_single = data['tau_off_slow'][0]
#                 time_on_double = data['time_on'][0]
#                 if time_on_double == 0:
#                     print('time_on: ' + date + '/' + plate + '/' + cell + '/' + str(i).zfill(3) + '.mat')
#                 time_off_double = data['time_off'][0]
#                 if time_off_double == 0:
#                     print('time_off: ' + date + '/' + plate + '/' + cell + '/' + str(i).zfill(3) + '.mat')

#                 amp_on_fast.append(amp_on_fast_single)
#                 amp_on_slow.append(amp_on_slow_single)
#                 amp_off_fast.append(amp_off_fast_single)
#                 amp_off_slow.append(amp_off_slow_single)
#                 tau_on_fast.append(tau_on_fast_single)
#                 tau_on_slow.append(tau_on_slow_single)
#                 tau_off_fast.append(tau_off_fast_single)
#                 tau_off_slow.append(tau_off_slow_single)
#                 time_on.append(time_on_double)
#                 time_off.append(time_off_double)

#             data_all['F0'].append(F0)
#             data_all['Leak'].append(leaks)
#             data_all['Ra'].append(Ra_s)
#             data_all['Leak_min'].append(np.min(leaks))
#             data_all['Ra_max'].append(np.max(Ra_s))

#             data_all['dFF'].append(dff)
#             data_all['amp_on_single'].append(amp_on)
#             data_all['amp_off_single'].append(amp_off)
#             data_all['tau_on_single'].append(tau_on)
#             data_all['tau_off_single'].append(tau_off)
#             data_all['time_on_10-90'].append(time_on)
#             data_all['time_off_10-90'].append(time_off)
#             data_all['bleach_x'].append(bleach_x)
#             data_all['bleach'].append(bleach)

#             data_all['amp_on_f_double'].append(amp_on_fast)
#             data_all['amp_on_s_double'].append(amp_on_slow)
#             data_all['amp_off_f_double'].append(amp_off_fast)
#             data_all['amp_off_s_double'].append(amp_off_slow)
#             data_all['tau_on_f_double'].append(tau_on_fast)
#             data_all['tau_on_s_double'].append(tau_on_slow)
#             data_all['tau_off_f_double'].append(tau_off_fast)
#             data_all['tau_off_s_double'].append(tau_off_slow)
# print('Done!')


In [ ]:
columns = ['Date','Plate','Cell','dFF_p','dFF_s']
df = pd.DataFrame(columns = columns)
df

In [ ]:
# dF/F0 data is in params / fit_pix_alpha_single / bkg_norm_ave_dFoF0_transient
# dF/F0 data is in params / fit_pix_alpha_single / bkg_norm_ave_on_dFoF0_plateau


data_all={}
for param in(df.columns):
    data_all[param] = []
for date in datePath:
    platePath = pb.getPathInfo(os.path.join(rawPath, date))
    for plate in platePath:
        cellPath = pb.getPathInfo(os.path.join(imgPath, date, plate))
        for cell in(cellPath):
            print(cell)
            matFiles = glob(os.path.join(imgPath, date, plate, cell) + '/*.mat')
            if len(matFiles) < 9:
                print('bad trial: ' + '/' + date + '/' + plate + '/' + cell)
                continue
            dFF_p = []
            dFF_s = []
            for trial in range(9):
                matFile = os.path.join(imgPath,date,plate,cell) + '/' + str(trial).zfill(3) + '.mat'
                f = h5py.File(matFile,'r')
                d = f['params']['fit_pix_alpha_single']
                st = d[0]
                obj = f[st]
                dff_p = obj['bkg_norm_ave_dFoF0_transient'][()][0]
                dff_s = obj['bkg_norm_ave_on_dFoF0_plateau'][()][0]
                dFF_p.append(dff_p)
                dFF_s.append(dff_s)
            data_all['Date'].append(date)
            data_all['Plate'].append('plate_000')
            data_all['Cell'].append(cell)
            data_all['dFF_p'].append(dFF_p)
            data_all['dFF_s'].append(dFF_s)
            
               
print('Done!')


In [ ]:
data_all['dFF_p']

In [ ]:
df_new = pd.DataFrame(data_all, columns = list(df))
df_new

In [ ]:
# df_new_1 = pd.DataFrame(data_all, columns = list(df)) # columns specification keeps the order of columns; index specification makes sure dict keys are the columns 
df = df.append(df_new, ignore_index = True)
df.index = np.arange(1, len(df)+1)
df


In [ ]:
# df = df.drop(df.index[len(df)-1]) # Drop the last row
# df

In [ ]:
df

In [ ]:
# df.to_pickle(sumPath + '/summary_sel.pkl')
# df.to_excel(sumPath + '/summary_sel.xlsx')

# df.to_pickle(rawPath + '/summary_3b_variants.pkl')
# df.to_excel(rawPath + '/summary_3b_variants.xlsx')

# df.to_pickle(rawPath + '/summary_Voltron_variants.pkl')
# df.to_excel(rawPath + '/summary_Voltron_variants.xlsx')


In [ ]:
## Check the size for each column is the same
# i = 0
# for k, v in data_all.items():
#     print(k, len(v))
#     i+=1


### Plot

- Filter data

In [ ]:
# df = pd.read_pickle(rawPath + '/summary_3b_variants.pkl')

In [ ]:
threshold = {
            'leak':-200,
            'Ra':30,

        }



In [ ]:
df = df_amy[(df.Leak_min>threshold['leak'])&(df.Ra_max<threshold['Ra'])]
# pd.set_option('display.max_columns', 50)
# pd.set_option('display.max_colwidth', 80)
print(len(df))
# df


- Plot dFF etc for each voltage step

In [ ]:
sensors = df.Mutation.unique()
# sensors = [ '376.20950','485.20950','376.13','376.20948','485.6',
#            '376.20949','485.20949', '376.17995']
sensors = ['ASAP3b.T392V.J','ASAP3b.T392V.E','ASAP3b.T392V.A', 'ASAP3b.T392V.B', 'ASAP3b.T392V.C',
       'ASAP3b.T392V.D',  'ASAP3b.T392V.F',
       'ASAP3b.T392V.G', 'ASAP3b.T392V.I', 'ASAP3b.T392V.H'
       ]
sensors

In [ ]:
# %matplotlib tk # plot in a new window
import unicodedata
import seaborn as sns
sns.set()
sns.set_style(style='whitegrid')

# fig = plt.figure(figsize=[12,13])
# params = ['dFF','amp_on_single','tau_on_single','tau_off_single','time_on_10-90','time_off_10-90']

# x_label = 'mV'
# y_label = ['dF/F','dF/F_amp_on','ms','ms','ms','ms']

fig = plt.figure(figsize=[6,5])
# params = ['amp_on_single']
params = ['dFF']
x_label = 'mV'
y_label = [unicodedata.lookup("GREEK CAPITAL LETTER delta") + 'F/F']
# labels = {
#           '376.1':'ASAP1',
#           '376.20948':'ASAP3b',
#           '376.20950':'ASAP3b T392V',
#           '376.13':'ASAP1 T392V',
#           '376.20949':'ASAP3b T392Y',
#           '485.6':'ASAP3b-SOM2',
#           '485.20949':'ASAP3b-SOM2 T392Y',
#           '485.20950':'ASAP3b-SOM2 T392V',
#           '376.18153':'ASAP1 N391I T392V',
#           '376.17995':'ASAP1 N391L T392I'

#          }


# labels = {
#             '421.1':'Voltron',
#             '421.2096':'Voltron A122D',
#             '421.4909':'Voltron N69E R78H A122D V196P',
#             '421.4919':'Voltron R78H A122D V196P',
#             '421.4876':'Voltron N69E A122D',
#             '421.4892':'Voltron N69E V196P'
    
#          }




labels = {
            'ASAP3b.T392V.A':'ASAP3b T392V-GSS3TR-SOM2',
            'ASAP3b.T392V.B':'ASAP3b KGC-GSS3-ER2-GSS3-SOM2',
            'ASAP3b.T392V.C':'GSSlong-SOM2',
            'ASAP3b.T392V.C':'ASAP3-Kv',
            'ASAP3b.T392V.D':'ASAP3-Kv(Lin)',
            'ASAP3b.T392V.E':'ASAP3b T392V-TS-ER-SOM2',
            'ASAP3b.T392V.F':'ASAP3b T392V-NL-SOM2',
            'ASAP3b.T392V.G':'ASAP3b T392V-lTlcnC',
            'ASAP3b.T392V.H':'ASAP3b T392V-NL-SOM2(Lin)',
            'ASAP3b.T392V.I':'ASAP3b T392V-GSS3-SOM2-lTlcnC',
            'ASAP3b.T392V.J':'ASAP3b T392V'
         }


for i in range(len(params)):
#     ax = plt.subplot(3,2,i+1)
    ax = plt.subplot(1,1,i+1)
#     for sensor in sensors:
#         
#             continue
    for sensor in sensors: # Manually pick up sensors to plot
        if sensor == '414.1':
            continue
    
        df_sensor = df[df.Mutation == sensor]
        param_mean = np.nanmean(df_sensor[params[i]].tolist(), axis=0)
        param_sem = np.nanstd(df_sensor[params[i]].tolist(), axis=0)/(len(df_sensor)-1)
        x = np.linspace(-100,60,9)
        if sensor in('ASAP3b.T392V.J','ASAP3b.T392V.E'):
#         if sensor in('376.20950','485.20950','376.20948'):
        
            ax.errorbar(x, param_mean, yerr = param_sem, fmt = 'o-',
                        label = labels[sensor]+' (%d)' %len(df_sensor))
        else:
            ax.errorbar(x, param_mean, yerr = param_sem, fmt = 'o-',
                        label = labels[sensor] +' (%d)' %len(df_sensor), alpha=0.2)

    ax.legend()
    ax.set_xlabel(x_label)
#     ax.set_ylim([-0.3,0.3])
    ax.set_ylabel(y_label[i])
#     ax.set_title(params[i])


handles, labels = ax.get_legend_handles_labels() # Following 3 lines remove the errorbars from legend
handles = [h[0] for h in handles]
ax.legend(handles, labels, loc='lower left', bbox_to_anchor=(0.55, 0.4))

plt.setp(ax, xticks=x)
plt.tight_layout()
# plt.legend()
# fig.savefig('/Users/zhengj10/Desktop/ASAP3b_variants_dFF.png',dpi=1200)
plt.show()


In [ ]:
# df.index = np.arange(1, len(df)+1)
# df.to_excel('/Users/zhengj10/Desktop//summary.xlsx')

In [ ]:
# df[df.Mutation == '376.18153']

- Plot kinetics at a certain voltage step

In [ ]:
fig = plt.figure(figsize=(10,4))

# params = ['tau_on_single','tau_off_single','tau_on_f_double','tau_on_s_double','tau_off_f_double','tau_off_s_double'
#          ,'time_on_10-90','time_off_10-90', 'F0']

params = ['tau_on_f_double','tau_off_f_double','tau_on_s_double','tau_off_s_double']
# params = ['tau_on_single','tau_off_single']

for f in range(len(params)):
    ax = plt.subplot(2,2,f+1)
    mean_all = []
    sem_all = []
    for sensor in sensors:
        df_sensor = df[df.Mutation == sensor]
        param_mean = np.nanmean(df_sensor[params[f]].tolist(), axis=0)
        param_mean = param_mean[6] # Index is the voltage step (e.g. 6 means 20mV)
        param_sem = np.nanstd(df_sensor[params[f]].tolist(), axis=0)/(len(df_sensor)-1)
        param_sem = param_sem[6] # Index is the voltage step (e.g. 6 means 20mV)
        mean_all.append(param_mean)
        sem_all.append(param_sem)
#     if params[f] in['tau_off_s_double']:
#         ax.set_ylim([0,800])
    

    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    y_pos = np.arange(1, len(sensors)+1)
    if f>1:
#         plt.xticks(y_pos, sensors, rotation=45, ha = 'right')
        plt.xticks(y_pos, labels, rotation=30, ha = 'right')
#     colors = ['k', 'y', 'm', 'c', 'g', 'r', 'b', 'brown']
    else:
        plt.xticks([]) 
#     colors = ['m', 'c', 'g', 'r', 'b', 'k', 'y', 'brown']
#     for xtick, color in zip(ax.get_xticklabels(), colors):
#         xtick.set_color(color)

    plt.tick_params(
        axis='x',          # changes apply to the x-axis
        which='both',      # both major and minor ticks are affected
        bottom=False,      # ticks along the bottom edge are off
        top=False,         # ticks along the top edge are off
        labelbottom=True)


    ax.set_ylabel(params[f][:-7]+' (ms)', size=12, weight='light')
    barlist = ax.bar(y_pos, mean_all, yerr = sem_all)
#     barlist[0].set_color('m')
#     barlist[1].set_color('c')
    # barlist[2].set_color('m')
    # barlist[3].set_color('c')
    # barlist[4].set_color('g')
    # barlist[5].set_color('r')
    # barlist[6].set_color('b')
    # barlist[7].set_color('brown')
    # plt.rcParams["axes.labelweight"] = "bold"
plt.tight_layout()
# fig.savefig('/Users/zhengj10/Desktop/ASAP3b_variants_kinetics.png',dpi=1200)
plt.show()


- dFF vs. F0

In [ ]:
# # sensor_to_plot = []
# plt.figure(figsize=[10,9])
# sns.set()
# i=1
# for sensor in sensors:
#     df_sensor = df[df.Mutation == sensor]
#     x = [cell[6] for cell in df_sensor['F0'].tolist()]
#     y = [cell[6] for cell in df_sensor['dFF'].tolist()]
#     ax = plt.subplot(3,3,i)
#     s_scatter = sns.scatterplot(x,y,ax=ax)
#     s_scatter.set_xlabel('')
#     s_scatter.set_ylabel(sensor, size=13)
#     s_scatter.set_xlabel('F0', size=13)
#     i+=1
# plt.tight_layout()
# # plt.savefig('/Users/zhengj10/Desktop/dFF_vs_F0.png')
# plt.show()


- Plot with raw data (self-defined ROI)

In [ ]:
def get_dff(imgData):
    dff_list = []
    f0 = np.mean(imgData[100:200, int(imgData.shape[1]/2)-20:int(imgData.shape[1]/2)+20, :])
    for frame in range(imgData.shape[0]):
        fb = np.mean(imgData[frame,:20,:20])
#         fb = np.min(imgData[frame,:,:])
        f = np.mean(imgData[frame, int(imgData.shape[1]/2)-20:int(imgData.shape[1]/2)+20, :])
        dff = (f-f0)/(f0-fb)
        dff_list.append(dff)
    return dff_list


In [ ]:
filePath = rawPath + '/2019.10.17_000/plate_002/patch_attempt_0025_000/camera_clamp2_000/'


print(os.path.exists(filePath))
dff_single_cell = []
for trial in range(9):

    dataFile = filePath + str(trial).zfill(3) + '/Camera/frames.ma'
    data = h5py.File(dataFile)
    imgData = data['data'][()]
    
    dff_list = get_dff(imgData)
    dff_single_cell.append(np.mean(dff_list[800:1200]))
# if dff_single[-1] < -0.3:
#     print(cell)
# dff_all.append(dff_single)


In [ ]:
data = h5py.File(dataFile)
imgData = data['data'][()]
# print(imgData.shape)

In [ ]:
plt.plot(dff_single_cell)
plt.show()


- Group plot

In [ ]:
# fig = plt.figure()
# dff_all_mean = np.mean(dff_all, axis=0)
# dff_all_sem = np.std(dff_all, axis = 0)/(len(dff_all)-1)
# x = np.linspace(-100,40,9)
# plt.errorbar(x, dff_all_mean, yerr=dff_all_sem, fmt='o-', markersize='8', lw=2)
# # fig.savefig('/Users/zhengj10/Desktop/414.1.png')
# plt.show()


In [ ]:
dff_all_sensor['414.1'] = dff_all_mean

In [ ]:
dff_all_sensor

In [ ]:
fig = plt.figure(figsize=[8,8])
# colormap = {'376.1':'#1f77b4', '376.13': '#ff7f0e', '414.1': '#2ca02c'}
for sensor in['376.1','376.13','414.1']:
    dff_all_mean = dff_all_sensor[sensor]
    dff_all_sem = np.std(dff_all, axis = 0)/(len(dff_all)-1)
    x = np.linspace(-100,40,9)
    plt.errorbar(x, dff_all_mean, yerr=dff_all_sem, fmt='o-', markersize='8', lw=2, label=sensor)
    plt.legend()
# fig.savefig('/Users/zhengj10/Desktop/SfN/PatcherBot.png', dpi=1200)
plt.show()


### Check meta data

In [ ]:
# dataFile = '/Users/zhengj10/Desktop/test/2019.10.17_000/plate_002/patch_attempt_0032_000/camera_clamp2_002/002/Clamp2.ma' # I and V steps
# dataFile = '/Users/zhengj10/Desktop/0925_376.1/patch_attempt_0000_000/camera_clamp2_000/000/LEDChannels.ma'
# dataFile = '/Users/zhengj10/Desktop/0925_376.1/patch_attempt_0000_000/camera_clamp2_000/000/Camera/daqResult.ma'
dataFile = '/Volumes/genie/Jihong_GENIE_stuff/PatcherBot/raw/2019.10.22_000/plate_002/patch_attempt_0025_000/camera_clamp2_000/000/Clamp2.ma'


In [ ]:
data = h5py.File(dataFile)
meta = data['data'][()]


In [ ]:
meta.shape


In [ ]:
fig = plt.figure(figsize=[10,12])
for ch in range(meta.shape[0]):
    ax = fig.add_subplot(3,1,ch+1)
    ax.plot(meta[ch,:])
    ax.set_xlim([19000,21000])
    if ch ==1:
        ax.set_ylim([-3*1e-9,0.3*1e-9])

plt.show()
# fig.savefig('/Users/zhengj10/Desktop/Clamp2.ma.data.2.png')


### Get tiff sequences for specified voltage step

In [ ]:
# dataFile = '/Users/zhengj10/Desktop/0925_376.1/patch_attempt_0000_000/camera_clamp2_000/008/Camera/frames.ma'
# dataFile = '/Users/zhengj10/Desktop/PatcherBot/2019.10.04_000/plate_001//patch_attempt_0008_000/camera_clamp2_000/008/Camera/frames.ma'
# dataFile = '/Volumes/genie/Jihong_GENIE_stuff/PatcherBot/patch_attempt_0035_000/camera_clamp2_000/008/Camera/frames.ma'
dataFile = os.path.join(rawPath,'2019.10.17_000/plate_002/patch_attempt_0025_000/camera_clamp2_000/000/Camera/frames.ma')

In [ ]:
dataFile

In [ ]:
data = h5py.File(dataFile)
imgData = data['data'][()]

In [ ]:
x = data['info']['0']['values'].value # Frame time stamp
x

In [ ]:
plt.plot(x[:10])
plt.show()


In [ ]:
dff_list = get_dff(imgData)
# dff_list

In [ ]:
plt.plot(x,dff_list, lw=0.3)
plt.show()


In [ ]:
# imsave('/Users/zhengj10/Desktop/test.tif', imgData)

In [ ]:
imgData.shape

### Save as one tif file (large size)

In [ ]:
entire_seq = np.zeros([2,508,41])
for trial in range(9):

#     dataFile = '/Users/zhengj10/Desktop/PatcherBot/2019.10.04_000/plate_001/patch_attempt_0025_000/camera_clamp2_000/' + str(trial).zfill(3) + '/Camera/frames.ma'
    data = h5py.File(dataFile)
    imgData = data['data'][()]
    print(imgData.shape)
    entire_seq = np.concatenate((entire_seq, imgData), axis=0)

entire_seq = entire_seq[2:,:,:]
entire_seq.shape


In [ ]:
# imsave('/Users/zhengj10/Desktop/test.tif', entire_seq)

In [ ]:
mean_img = np.mean(entire_seq, axis=0)
mean_img.shape
np.max(mean_img)-np.min(mean_img)


In [ ]:
plt.figure(figsize=[16,10])
plt.imshow(mean_img, cmap='gray')
# plt.colorbar()
plt.show()


### Read all variables in mat file with h5py

In [ ]:
# f = h5py.File(matFile)
# f = f['params']
# variables = f.items()

# for var in variables:
#     name = var[0]
#     data = var[1]
#     print( "Name ", name)  # Name
#     if type(data) is h5py.Dataset:
#         # If DataSet pull the associated Data
#         # If not a dataset, you may need to access the element sub-items
#         value = data.value
#         print("Value", value)  # NumPy Array / Value